In [1]:
# Libraries
import numpy as np
from numpy import loadtxt
import csv
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import sklearn
import random

from sklearn.linear_model import LogisticRegression

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Import
path = r'data/problem_charlson_death.csv'
problem_charlson_death = pd.read_csv(path)
# Import
path = r'data/trimmed_patient_embedding_128d.csv'
patients_embeddings = pd.read_csv(path, index_col=0)
# Import
path = r'data/trimmed_patient_embedding_128d_random.csv'
patients_embeddings_random = pd.read_csv(path, index_col=0)

# Fill in nan
problem_charlson_death.death_year_label.fillna(0, inplace=True)
problem_charlson_death.death_label.fillna(0, inplace=True)
# Merge
patients_embeddings = pd.concat([problem_charlson_death[['death_year_label', 'death_label']], patients_embeddings], axis=1)
patients_embeddings_random = pd.concat([problem_charlson_death[['death_year_label', 'death_label']], patients_embeddings_random], axis=1)
# Drop na
patients_embeddings = patients_embeddings.dropna()
# Import
path = r'data/final_problem_dummies.csv'
problem_dummies = pd.read_csv(path)
# Drop columns
problem_dummies.drop(columns=['SUBJECT', 'PROBLEM_DT_TM'], inplace=True)
# Remove prefix
problem_dummies.columns = problem_dummies.columns.str.strip('PROBLEM_')
# Set index
problem_dummies.set_index('new_subject', inplace=True)
# Merge
problem_dummies = pd.concat([problem_charlson_death[['death_year_label', 'death_label']], problem_dummies], axis=1)
# Import
path = r'data/final_trimmed_snomed_embedding_128d.csv'
snomed_embeddings = pd.read_csv(path, index_col=0)
# Get list 
snomed_embeddings.index = snomed_embeddings.index.astype(str)
snomed_list = snomed_embeddings.index.tolist()
problem_dummies_list = problem_dummies.columns.tolist()
overlap_list = list(set(snomed_list) & set(problem_dummies_list))
overlap_list = ['death_year_label', 'death_label'] + overlap_list
# Get index / patient list
patients_embeddings_list = patients_embeddings.index.tolist()
# Filter
problem_dummies = problem_dummies[overlap_list]
problem_dummies = problem_dummies.loc[patients_embeddings_list]
patients_embeddings_random = patients_embeddings_random.loc[patients_embeddings_list]
# Work out count for problems
problem_sum = pd.DataFrame(problem_dummies.iloc[:,2:].sum(axis=0), columns=['Count']).sort_values(by=['Count'], ascending=False)

In [ ]:
# Save
#problem_dummies.to_csv('slim_problem_dummies_death.csv')